# Análise de Fatores de Risco para AVC
## 1. Introdução
- Breve descrição do problema e objetivos do projeto.
- Fonte dos dados (link e descrição).
- Tecnologias utilizadas.

## 2. Importação das Bibliotecas e do Dataset

In [1]:
import pandas as pd
import matplotlib as plt

## 3. Entendimento Inicial dos Dados
- shape, head(), info(), describe()
- Verificação de tipos de dados, valores únicos, nulos, etc.

In [3]:
#importando dados do arquivo raw
url = 'https://raw.githubusercontent.com/danoliveiraed/portfolio-projects/refs/heads/main/stroke-risk-analysis/data/raw/healthcare-dataset-stroke-data.csv'

dado = pd.read_csv(url, sep=',')
dado.head(3)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1


In [4]:
dado.tail(3)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0
5109,44679,Female,44.0,0,0,Yes,Govt_job,Urban,85.28,26.2,Unknown,0


In [11]:
#verificando tipos dos dados
dado.shape
dado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [6]:
#verificando dados nulos
dado.isnull().sum()

,0
id,0
gender,0
age,0
hypertension,0
heart_disease,0
ever_married,0
work_type,0
Residence_type,0
avg_glucose_level,0
bmi,201


In [7]:
#verificando dados duplicados
dado.duplicated().sum()

np.int64(0)

In [14]:
dado.count()

,0
id,5110
gender,5110
age,5110
hypertension,5110
heart_disease,5110
ever_married,5110
work_type,5110
Residence_type,5110
avg_glucose_level,5110
bmi,4909


## 4. Limpeza e Pré-processamento
- Tratamento de valores ausentes (ex: `bmi`)
- Conversão de tipos e renomeação de colunas
- Padronização de categorias
- Criação de novas variáveis, se necessário

In [16]:
#prencher bmi com média por gênero
dado['bmi'] = dado.groupby('gender')['bmi'].transform(
    lambda x: x.fillna(x.mean())
)
dado.isnull().sum()

,0
id,0
gender,0
age,0
hypertension,0
heart_disease,0
ever_married,0
work_type,0
Residence_type,0
avg_glucose_level,0
bmi,0


## 5. Análise Exploratória (EDA)
### 5.1 Perfil demográfico e clínico
- Distribuição de idade, gênero, IMC, glicose
- Frequência de hipertensão, doenças cardíacas

### 5.2 Comparações com a variável alvo (stroke)
- Taxa de AVC por grupo etário, sexo, hipertensão etc.
- Gráficos de comparação (barplots, boxplots, etc.)

## 6. Agregações para Visualização no Power BI
- Exportar CSVs com:
  - AVC por idade/grupo etário
  - AVC por hipertensão, glicose, IMC (categorizado)
  - Tabelas de cruzamento por gênero, tipo de trabalho, estado civil

## 7. Conclusões e próximos passos
- Sumário de insights
- Oportunidades de visualização no Power BI

# 8. Salvando arquivos para o Power BI
- `to_csv()` dos dataframes agregados